In [73]:
#сперва нужно хотя бы понять, кого мы хотим нанимать
#для этого просто воспользуемся здравым смыслом, посмотрим вакансии на птицефабриках, фермах и тд. на hh.ru

In [74]:
#вот основные сотрудники, которые нам нужны
#зоотехник - отвечает за куриц, их корм и оптимальные условия содержания
#ветеринар - вдруг куры заболеют
#водитель - развоз кур клиентам
#инженер-механик - отвечает за установку, тех. осмотр и ремонт фермы
#разнорабочий - выполняет всю необходимую дополнительную физическую работу

In [75]:
#далее мы хотим назначить для каждого сотрудника оптимальную зарплату
#здесь важно уточнить, что мы планируем работать в 5 крупнейших городах России: Москва, Санкт-Петербург, Новосибирск, Екатеринбург и Казань
#и понятно, что оптимальная зарплата будет отличаться для каждого города, поэтому правильнее собирать данные в отдельности по каждому городу
#для этого будем искать открытые вакансии на hh.ru и Telegram-канале "какой-то канал"
#спарсим эти источники, результатом будет некоторый датафрейм с названием вакансии в одной колонке и предлагаемой зарплатой в другой

In [76]:
# напишем парсер для hh.ru, который будет искать вакансии по ключевым словам - названиями должностей

In [77]:
#просто импорт необходимых штук
import requests
import pandas as pd

def get_hh_vacancies(key_words, area, pages = 5, only_with_salary = True):
#пояснения к параметрам функции:
#это параметры к api
#key_words - ключевые слова, по которым ищем вакансию
#area - регион, где мы ищем вакансию
#pages - количество страниц, которых мы хотим просмотреть(значение по умолчанию = 5)
#only_with_salary - фильтр для тех вакансий, где явно указана зарплата(значение по умолчанию = True)
    base_url = "https://api.hh.ru/vacancies" # базовый API HH.ru

    vacancies = [] #сюда будем записывать данные

    for page in range(pages):
        params = {"text": key_words, "area": area, "page": page, "only_with_salary": only_with_salary}
        response = requests.get(base_url, params=params)#получили наш HTTP-ответ от API

        if response.status_code != 200:
          print(f"Ошибка! Причина: {response.text}")  #проверяет, что запрос произршёл успешно
          break

        data = response.json()#переводим ответ в формат json, чтобы дальше извлечь необохимые данные
        vacancies.extend(data.get("items", []))
        #data.get("items", []) возвращает значение для ключа "items"
        #метод extend добавляет в конец списка информацию по каждой вакансии
        #то есть фактически vacancies сейчас представляет собой, список словарей
        #где в каждом словаре записаны характеристики каждой вакансии

        if page >= data["pages"] - 1: #проверка на достижение последней страницы
            break

    results = []
    for vacancy in vacancies:#проходимся по каждой вакансии
        salary = vacancy.get("salary") #достаём содержимое в ячейке salary
        #поскольку зачастую работодатели указывают вилку зп, а не конкретное значение,
        #то это надо учитывать
        real_salary = 0
        if salary:
          if salary["from"] and salary["to"]: #если указаны обе границы, то берём среднее арифм.
            real_salary = (salary["from"] + salary["to"]) / 2
          else:#если указана лишь одна из ограниц, берём её
            real_salary = salary["from"] or salary["to"]


        results.append({
            "Вакансия": vacancy["name"],
            "Зарплата": real_salary,
            "Регион": area
        })

    return pd.DataFrame(results)



In [78]:
#воспользуемся данными о id регионов, которые мы тоже получили с помощью api hh.ru

cities = {'Город':['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань'], 'ID':[1, 2, 4, 3, 88]}
df_cities = pd.DataFrame(cities)

In [79]:
df_cities

,Город,ID
0,Москва,1
1,Санкт-Петербург,2
2,Новосибирск,4
3,Екатеринбург,3
4,Казань,88


In [84]:
#пример использования функции
profession = 'зоотехник'
df_farm = get_hh_vacancies(profession,4,15)
print(f"\nНайдено вакансий для специализации {profession}: {len(df_farm)}")
df_farm


Найдено вакансий для специализации зоотехник: 6


,Вакансия,Зарплата,Регион
0,Специалист по уходу за аквариумами,50000.0,4
1,Зоотехник,64000.0,4
2,Главный зоотехник,143000.0,4
3,Главный зоотехник (КРС),150000.0,4
4,Зоотехник - осеменатор,80000.0,4
5,Зоотехник-селекционер,60000.0,4
